In [1]:
!pip install scraibe chromadb ollama sentence_transformers

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import warnings
warnings.filterwarnings('ignore')

### Settings

In [27]:
# audio_file = 'deshields_interview-medium.mp3'
audio_file = "deshields_interview.mp3"

## Transcribe and diarize audio from input file

In [28]:
from scraibe import Scraibe

model = Scraibe()
text = model.autotranscribe(audio_file)

In [29]:
text.to_json("full_transcription.json")

In [5]:
text.get_dict()

{0: {'speakers': 'SPEAKER_00',
  'segments': [0.58784375, 22.12034375],
  'text': " Okay, go ahead, Aaliyah. Today is July 26th, Tuesday. It is 10 22 a.m. This is Aaliyah Parsons here with Michael Sharappa, Marcus Smith and Doreen Rogers. We're conducting an interview with Rowena DeShield that will focus on her life history and observations in the village of Bellevue, Maryland."},
 1: {'speakers': 'SPEAKER_02',
  'segments': [23.63909375, 28.39784375],
  'text': ' Very good. Would you like to do the first question?'},
 2: {'speakers': 'SPEAKER_00',
  'segments': [28.296593750000003, 28.836593750000002],
  'text': ' Sure.'},
 4: {'speakers': 'SPEAKER_00',
  'segments': [29.27534375, 34.92846875],
  'text': " Um, let's start. If you could just state who you are and when and where you were born."},
 5: {'speakers': 'SPEAKER_01',
  'segments': [36.05909375, 46.70721875],
  'text': " I am Ruina Shields, a native of Bellevue. I was born on October 15, 1930 in Bellevue near my grandfather's h

In [34]:
print(text)

SPEAKER_06 (00:00:00 ; 00:00:02):	 Okay, go ahead, go ahead.
SPEAKER_02 (00:00:02 ; 00:00:22):	 Today is July 26th, Tuesday. It is 1022 a.m. This is Aliyah Parsons here with Michael Sharappa, Marcus Smith, and Dorian Rogers. We are conducting an interview with Rowena DeShields that will focus on her life history and observations in the village of Bellevue, Maryland.
SPEAKER_06 (00:00:23 ; 00:00:28):	 Very good. Would you like to do the first question?
SPEAKER_02 (00:00:28 ; 00:00:34):	 Sure. Let's start. If you could just state who you are and when and where you were born.
SPEAKER_05 (00:00:36 ; 00:00:46):	 I am Rowena Shields, a native of Bellevue. I was born November 15, 1930 in Bellevue near my grandfather's house.
SPEAKER_06 (00:00:47 ; 00:00:49):	 What was that date again?
SPEAKER_05 (00:00:49 ; 00:00:51):	 November 15, 1930.
SPEAKER_06 (00:00:49 ; 00:00:52):	 November 15, 1930.
SPEAKER_05 (00:00:52 ; 00:00:53):	 in Bellevue, Maryland.
SPEAKER_02 (00:00:55 ; 00:01:00):	 Who were y

In [33]:
text.transcript[0]

{'speakers': 'SPEAKER_06',
 'segments': [0.60471875, 2.25846875],
 'text': ' Okay, go ahead, go ahead.'}

## Extract and save audio of the interviewee

In [35]:
import IPython

from pydub import AudioSegment

SPEAKER = "SPEAKER_05"  # manually selected by Tim, could do programmatically based on length of speech?
audio = AudioSegment.from_mp3(audio_file)

# Retrieve timestamps from when that speaker is talking
timestamps = []
for index in text.transcript:
    entry = text.transcript[index]
    if entry['speakers'] == SPEAKER:
        start, end = entry['segments'][0]*1000, entry['segments'][1]*1000
        timestamps.append((start, end))

# Concatenate those timestamps as an audio file
extracted_audio = AudioSegment.empty()
for start, end in timestamps:
    extracted_audio += audio[start:end]

output_file = "deshields_voice.mp3"
extracted_audio.export(output_file, format="mp3")
IPython.display.Audio(output_file)  # See Applio for then running training, inference using voice sample

KeyboardInterrupt: 

### Create LLM Q&A from transcript

In [36]:
# Retrieve a simplified text copy of the transcript (just speaker, line)

SPEAKER = "SPEAKER_05"  # manually selected by Tim, could do programmatically based on length of speech?
deshields_entries = []
all_entries = []

for index in text.transcript:
    entry = text.transcript[index]
    if entry['speakers'] == SPEAKER:
        deshields_entries.append(entry['text'])
    all_entries.append(entry['text'])

In [37]:
deshields_entries[:5]

[" I am Rowena Shields, a native of Bellevue. I was born November 15, 1930 in Bellevue near my grandfather's house.",
 ' November 15, 1930.',
 ' in Bellevue, Maryland.',
 " Oh, wow. Most of them are gone. When I was born, all of my parents were aunts, uncles, cousins, nephews, and we stayed at my grandfather's house. And then when we found out that they were developing this area, I purchased a home here. But beginning, I was mostly with my grandmother and aunt on Gate Street. Then I had relatives across the street in Poplar Lane, but we were all related.",
 ' But my father was Golden Boots Thomas.']

In [10]:
all_entries[:5]

[" Okay, go ahead, Aaliyah. Today is July 26th, Tuesday. It is 10 22 a.m. This is Aaliyah Parsons here with Michael Sharappa, Marcus Smith and Doreen Rogers. We're conducting an interview with Rowena DeShield that will focus on her life history and observations in the village of Bellevue, Maryland.",
 ' Very good. Would you like to do the first question?',
 ' Sure.',
 " Um, let's start. If you could just state who you are and when and where you were born.",
 " I am Ruina Shields, a native of Bellevue. I was born on October 15, 1930 in Bellevue near my grandfather's house."]

In [38]:
import chromadb
from chromadb.utils import embedding_functions
import ollama
import json


# Create ChromaDB
client = chromadb.PersistentClient(path="./chromadb_store")
collection_name = "deshields-test"
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
collection = client.get_or_create_collection(name=collection_name, embedding_function=embedding_func)

# Ingest data
data = deshields_entries
for idx, chunk in enumerate(data):
    collection.add(documents=[chunk], ids=[str(idx)])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [43]:
# Create user query
question = "Who made their own eel traps?"

# Query database based on user query
results = collection.query(query_texts=[question], n_results=10)

# Extract context and format it properly
context_list = results.get("documents", [[]])[0]  # Handle missing key gracefully
context_text = "\n\n".join(context_list) if context_list else "No relevant context found."

# Handle empty context case
if not context_list:
    print("No relevant context found. Unable to answer the question.")
    exit()

# Define messages for LLM chat
messages = [
    {
        "role": "system",
        "content": (
            "You are an AI assistant answering questions about a historical interview with Rowena DeShields. "
            "The interview has transcription errors, and your job is to clean up mistakes while staying accurate. "
            "If the context does not contain enough information to answer the question, say so."
        ),
    },
    {"role": "user", "content": f"Context:\n{context_text}\n\nQuestion:\n{question}\n\nAnswer:"},
]

# Query LLM
response = ollama.chat(model="llama3.1", messages=messages)

# Display results
print("QUESTION:", question, "\n")
print("CONTEXT:", context_text, "\n")
print("RESPONSE:", response['message']['content'], "\n")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


QUESTION: Who made their own eel traps? 

CONTEXT: [" And then he'd fish in line. He'd just sit at the back of his house. His eel paws, fish in line. He'd catch his own eels. And he would cut them up, and then he would go down and bait his line.", ' all animals, especially rabbits, geese. He made, he roasted a crabber, but he made his own eel pods because he quickly yields himself. He made the pods, he made his own fishing line. He was just self-sufficient, but not to be educated.', ' where they cook.', " Because Kenny Turner, the smallest one of the Turner family built the first sailboat around here. He built a go-kart out of a refrigerator loader. They're just natural.", ' No, no, no, they do say he was in the boat building. He went around a lot. He dealt with the fishermen and he knew people in Tillman Island, all of those.', ' But we cracked claws to her body, but mother did another part of the craft.', ' did not go back to take the, I think they were stored in the attic. But they 